# Lecture 17 – Collision theory

In [ ]:
%pip install -q ampform ipympl ipywidgets matplotlib sympy
try:
    from google.colab import output

    output.enable_custom_widget_manager()
except:
    pass

In [ ]:
%matplotlib widget
from __future__ import annotations

import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import UnevaluatedExpression, implement_doit_method
from ampform.sympy.math import NumPyPrintable, create_expression
from IPython.display import Math
from matplotlib.backend_bases import Event
from matplotlib.colors import Normalize
from sympy.printing.numpy import NumPyPrinter
from sympy.printing.printer import Printer
from sympy.printing.pycode import PythonCodePrinter

_This notebook is an attempt to recreate the Mathematica notebook [provided by Miguel Albaladejo](https://indico.ific.uv.es/event/6803/contributions/21224)._

## Riemann sheets

### Square root example

There are multiple solutions for $x$ to the equation $y^2 = x$ – the fact that we usually take $y = \sqrt{x}$ to be the solution to this equation is just a matter of convention. It would be more complete to represent the solution as a set of points in the complex plane. In this case, we have the set $S = \left\{\left(z, w\right)\in\mathbb{C}^2 | w^2=z\right\}$. This is set forms a **Riemann surface** in $\mathbb{C}^2$ space.

In [ ]:
def plot_sqrt_sheets(f1, f2) -> None:
    r = np.linspace(0, 1, num=20)
    theta_n = np.linspace(-np.pi, 0, num=30)
    theta_p = np.linspace(0, +np.pi, num=30)
    R, Θn = np.meshgrid(r, theta_n)
    R, Θp = np.meshgrid(r, theta_p)
    Xn = R * np.cos(Θn)
    Xp = R * np.cos(Θp)
    Yn = R * np.sin(Θn)
    Yp = R * np.sin(Θp)
    Zn = Xn + Yn * 1j
    Zp = Xp + Yp * 1j

    fig, axes = plt.subplots(figsize=(9, 4), ncols=2, subplot_kw={"projection": "3d"})
    im_ax, re_ax = axes
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    for ax in axes:
        ax.set_xlabel("Re $z$")
        ax.set_ylabel("Im $z$")
        ax.set_xticks([-1, 0, +1])
        ax.set_yticks([-1, 0, +1])
        ax.set_zticks([-1, 0, +1])
    im_ax.set_title(R"Im $\sqrt[\pm]{z}$")
    re_ax.set_title(R"Re $\sqrt[\pm]{z}$")

    # We have to plot several surfaces to nicely render the cut
    im_ax.plot_surface(Xn, Yn, f1(Zn).imag, facecolors=plt.cm.coolwarm(f1(Zn).real))
    im_ax.plot_surface(Xn, Yn, f2(Zn).imag, facecolors=plt.cm.coolwarm(f2(Zn).real))
    im_ax.plot_surface(Xp, Yp, f1(Zp).imag, facecolors=plt.cm.coolwarm(f1(Zp).real))
    im_ax.plot_surface(Xp, Yp, f2(Zp).imag, facecolors=plt.cm.coolwarm(f2(Zp).real))
    re_ax.plot_surface(Xn, Yn, f1(Zn).real, facecolors=plt.cm.coolwarm(f1(Zn).imag))
    re_ax.plot_surface(Xn, Yn, f2(Zn).real, facecolors=plt.cm.coolwarm(f2(Zn).imag))
    re_ax.plot_surface(Xp, Yp, f1(Zp).real, facecolors=plt.cm.coolwarm(f1(Zp).imag))
    re_ax.plot_surface(Xp, Yp, f2(Zp).real, facecolors=plt.cm.coolwarm(f2(Zp).imag))

    sm = plt.cm.ScalarMappable(cmap=plt.cm.coolwarm, norm=Normalize(-1, +1))
    sm.set_array([])
    cb = plt.colorbar(sm, ax=re_ax, pad=0.15, shrink=0.6)
    cb.ax.set_yticks([-1, 0, +1])
    cb.ax.set_xlabel(R"Im $\sqrt[\pm]{z}$")
    cb = plt.colorbar(sm, ax=im_ax, pad=0.15, shrink=0.6)
    cb.ax.set_yticks([-1, 0, +1])
    cb.ax.set_xlabel(R"Re $\sqrt[\pm]{z}$")
    plt.tight_layout()
    fig.canvas.mpl_connect("motion_notify_event", sync_axes(fig, im_ax, re_ax))
    plt.show()


def sync_axes(fig, ax1, ax2) -> None:
    def on_move(event: Event) -> None:
        if event.inaxes == ax1:
            ax2.view_init(elev=ax1.elev, azim=ax1.azim)
        elif event.inaxes == ax2:
            ax1.view_init(elev=ax2.elev, azim=ax2.azim)
        else:
            return
        fig.canvas.draw_idle()

    return on_move

In [ ]:
plot_sqrt_sheets(
    f1=lambda z: -np.sqrt(z),
    f2=lambda z: +np.sqrt(z),
)

A subset of this Riemann sheet is defined by the following function, which has only a single sheet structure for its real part.

In [ ]:
@implement_doit_method
class SignedSqrt(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, z, sign, **hints) -> SignedSqrt:
        return create_expression(cls, z, sign, **hints)

    def evaluate(self) -> sp.Expr:
        z, sign = self.args
        return sp.sqrt(abs(z)) * sp.exp(sign * sp.I * sp.arg(z) / 2)

    def _latex(self, printer: LatexPrinter, *args) -> str:
        z = printer._print(self.args[0])
        sign = _render_sign(self.args[1], printer)
        return Rf"\sqrt[{sign}]{{{z}}}"


def _render_sign(sign, printer: Printer) -> str:
    if sign == +1:
        return "+"
    if sign == -1:
        return "-"
    return printer._print(sign)

In [ ]:
z = sp.Symbol("z")
neg_sqrt = SignedSqrt(z, sign=-1)
pos_sqrt = SignedSqrt(z, sign=+1)

In [ ]:
Math(aslatex({e: e.doit() for e in [neg_sqrt, pos_sqrt]}))

In [ ]:
plot_sqrt_sheets(
    f1=sp.lambdify(z, neg_sqrt.doit()),
    f2=sp.lambdify(z, pos_sqrt.doit()),
)

<details>
<summary>Video explainers</summary>

<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/R9MX8QgKwtg?si=kK-_1Po4XHzRpzR9" title="What are... Riemann surfaces? [VisualMath]" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

<iframe width="560" height="315" src="https://www.youtube.com/embed/sD0NjbwqlYw?si=XQMBxuwolPGlVbwq" title="Visualizing the Riemann zeta function and analytic continuation [3Blue1Brown]" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>
  
</details>